In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

In [2]:
# Define transform
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load the full CIFAR-10 dataset
full_trainset = torchvision.datasets.CIFAR10(root='./data/', train=True, download=True, transform=transform)
full_testset = torchvision.datasets.CIFAR10(root='./data/', train=False, download=True, transform=transform)

# Filter dataset: Exclude class 9
filtered_train_data_no_9 = [(img, label) for img, label in full_trainset if label != 9]
filtered_train_images_no_9, filtered_train_labels_no_9 = zip(*filtered_train_data_no_9)

# Filter dataset: Only class 9
filtered_train_data_only_9 = [(img, label) for img, label in full_trainset if label == 9]
filtered_train_images_only_9, filtered_train_labels_only_9 = zip(*filtered_train_data_only_9)

# Create dataset without class 9
trainset_no_9 = torch.utils.data.TensorDataset(
    torch.stack(filtered_train_images_no_9), torch.tensor(filtered_train_labels_no_9)
)
trainloader_no_9 = torch.utils.data.DataLoader(trainset_no_9, batch_size=4, shuffle=True)

# Create dataset with only class 9
trainset_only_9 = torch.utils.data.TensorDataset(
    torch.stack(filtered_train_images_only_9), torch.tensor(filtered_train_labels_only_9)
)
trainloader_only_9 = torch.utils.data.DataLoader(trainset_only_9, batch_size=4, shuffle=True)

# Filter testset: Exclude class 9
filtered_test_data_no_9 = [(img, label) for img, label in full_testset if label != 9]
filtered_test_images_no_9, filtered_test_labels_no_9 = zip(*filtered_test_data_no_9)

# Filter dataset: Only class 9
filtered_test_data_only_9 = [(img, label) for img, label in full_testset if label == 9]
filtered_test_images_only_9, filtered_test_labels_only_9 = zip(*filtered_test_data_only_9)

# Create test dataset without class 9
testset_no_9 = torch.utils.data.TensorDataset(
    torch.stack(filtered_test_images_no_9), torch.tensor(filtered_test_labels_no_9)
)
testloader_no_9 = torch.utils.data.DataLoader(testset_no_9, batch_size=4, shuffle=True)

# Create test dataset with only class 9
testset_only_9 = torch.utils.data.TensorDataset(
    torch.stack(filtered_test_images_only_9), torch.tensor(filtered_test_labels_only_9)
)
testloader_only_9 = torch.utils.data.DataLoader(testset_only_9, batch_size=4, shuffle=True)


# Normal trainloader (all classes)
trainloader_full = torch.utils.data.DataLoader(full_trainset, batch_size=4, shuffle=True)

# Normal testloader (unchanged)
testloader = torch.utils.data.DataLoader(full_testset, batch_size=4, shuffle=False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 
           'dog', 'frog', 'horse', 'ship', 'truck')

# Print dataset sizes
print(f"Original training set size: {len(full_trainset)}")
print(f"Training set size without class 9: {len(trainset_no_9)}")
print(f"Training set size with only class 9: {len(trainset_only_9)}")


Files already downloaded and verified
Files already downloaded and verified
Original training set size: 50000
Training set size without class 9: 45000
Training set size with only class 9: 5000


In [3]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 84)
        #self.fc4 = nn.Linear(84, 84)
        self.fc5 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x), inplace = True)
        x = F.relu(self.fc2(x), inplace = True)
        x = F.relu(self.fc3(x), inplace = True)
        #x = F.relu(self.fc4(x))
        x = self.fc5(x)
        
        return x
    
net = Net()

In [9]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [10]:
import os

# Define paths
model_directory_path = 'model/'
model_path = os.path.join(model_directory_path, 'cifar-mnist-cnn-model.pt')

# Ensure directory exists
if not os.path.exists(model_directory_path):
    os.makedirs(model_directory_path)

def load_model(net):
    """Loads the trained model if it exists."""
    if os.path.isfile(model_path):
        net.load_state_dict(torch.load(model_path))
        print('Loaded model parameters from disk.')
        return True  # Indicate successful loading
    else:
        print('No saved model found.')
        return False

def train_model(net, trainloader, criterion, optimizer, epochs=2):
    """Trains the model from scratch."""
    print("Starting training...")
    for epoch in range(epochs):
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data

            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if i % 2000 == 1999:  # Print every 200 mini-batches
                print('[%d, %5d] loss: %.3f' %
                    (epoch + 1, i + 1, running_loss / 2000))
                running_loss = 0.0
                
    print('Finished Training.')

In [11]:
def layer_training(net, trainloader, criterion, optimizer, epochs=2):

    # Freeze all layers initially
    for param in net.parameters():
        param.requires_grad = False
    
    #Unfreeze single layer
    for param in net.fc1.parameters():
        param.requires_grad = True

    # Start training
    for epoch in range(epochs):
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data

            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if trainloader == trainloader_only_9:
                if i % 200 == 199:  # Print every 200 mini-batches
                    print('[%d, %5d] loss: %.3f' %
                          (epoch + 1, i + 1, running_loss / 2000))
                    running_loss = 0.0
            elif trainloader == trainloader_full:
                if i % 2000 == 1999:  # Print every 2000 mini-batches
                    print('[%d, %5d] loss: %.3f' %
                          (epoch + 1, i + 1, running_loss / 2000))
                    running_loss = 0.0
    print('Finished Training.')

In [42]:
train_model(net, trainloader_no_9, criterion, optimizer, 1)

Starting training...
[1,  2000] loss: 0.904
[1,  4000] loss: 0.887
[1,  6000] loss: 0.885
[1,  8000] loss: 0.898
[1, 10000] loss: 0.880
Finished Training.


In [52]:
layer_training(net, trainloader_only_9, criterion, optimizer, 3)

[1,   200] loss: 0.223
[1,   400] loss: 0.218
[1,   600] loss: 0.216
[1,   800] loss: 0.216
[1,  1000] loss: 0.215
[1,  1200] loss: 0.213
[2,   200] loss: 0.212
[2,   400] loss: 0.209
[2,   600] loss: 0.210
[2,   800] loss: 0.210
[2,  1000] loss: 0.209
[2,  1200] loss: 0.208
[3,   200] loss: 0.206
[3,   400] loss: 0.205
[3,   600] loss: 0.203
[3,   800] loss: 0.204
[3,  1000] loss: 0.201
[3,  1200] loss: 0.200
Finished Training.


In [54]:
layer_training(net, trainloader_full, criterion, optimizer, 2)

[1,  2000] loss: 1.513
[1,  4000] loss: 1.294
[1,  6000] loss: 1.257
[1,  8000] loss: 1.246
[1, 10000] loss: 1.234
[1, 12000] loss: 1.212
[2,  2000] loss: 1.196
[2,  4000] loss: 1.195
[2,  6000] loss: 1.181
[2,  8000] loss: 1.178
[2, 10000] loss: 1.185
[2, 12000] loss: 1.176
Finished Training.


In [55]:
import numpy as np
total_correct = 0
total_images = 0
confusion_matrix = np.zeros([10,10], int)
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total_images += labels.size(0)
        total_correct += (predicted == labels).sum().item()
        for i, l in enumerate(labels):
            confusion_matrix[l.item(), predicted[i].item()] += 1 

model_accuracy = total_correct / total_images * 100
print('Model accuracy on {0} test images: {1:.2f}%'.format(total_images, model_accuracy))

Model accuracy on 10000 test images: 55.68%


In [56]:
total_correct_2 = 0
total_images_2 = 0
confusion_matrix = np.zeros([10,10], int)

with torch.no_grad():
    for data in testloader_no_9:
        images, labels = data

        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total_images_2 += labels.size(0)
        total_correct_2 += (predicted == labels).sum().item()
        
        for i, l in enumerate(labels):
            confusion_matrix[l.item(), predicted[i].item()] += 1 

no_9_accuracy = total_correct_2 / total_images_2 * 100
print('Model accuracy on no 9: {:.2f}%'.format(no_9_accuracy))

Model accuracy on no 9: 61.87%


In [57]:
total_correct_3 = 0
total_images_3 = 0
confusion_matrix = np.zeros([10,10], int)

with torch.no_grad():
    for data in testloader_only_9:
        images, labels = data

        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total_images_3 += labels.size(0)
        total_correct_3 += (predicted == labels).sum().item()
        
        for i, l in enumerate(labels):
            confusion_matrix[l.item(), predicted[i].item()] += 1 

only_9_accuracy = total_correct_3 / total_images_3 * 100
print('Model accuracy on only 9: {:.2f}%'.format(only_9_accuracy))

Model accuracy on only 9: 0.00%


In [59]:
print('Model accuracy on {0} test images: {1:.2f}%'.format(total_images, model_accuracy))
print('Model accuracy only 9s: {:.2f}%'.format(only_9_accuracy))
print('Model accuracy no 9s: {:.2f}%'.format(no_9_accuracy))

Model accuracy on 10000 test images: 55.68%
Model accuracy only 9s: 0.00%
Model accuracy no 9s: 61.87%


In [21]:
model = net
torch.save(model, 'model_unlucky_9_asdf_full.pth')